In [1]:
# import transformers

# print(transformers.__version__)

In [2]:
# !nvidia-smi

In [3]:
# # RUN THIS ON CPU
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"    
# import tensorflow as tf

In [4]:
# PATH_TO_FOLDER = "drive/MyDrive/Name2Demographics/"
PATH_TO_FOLDER = "/home/souvic/mounted/btp/vahini/Name2Demographics/"
%cd $PATH_TO_FOLDER

/home/souvic/mounted/btp/vahini/Name2Demographics


In [5]:
import sys  
sys.path.insert(0, 'Models/ERData/PreProcessing/')

from er_preprocess import *

ER Dataset

In [6]:
from datasets import Dataset, load_dataset
from transformers import TrainingArguments
from transformers import Trainer

In [15]:
# Extract the dataset
# er_df =  ERData_MajorityLabel()
er_df = ERData_with_dup()
# er_df['Name'].to_csv('Models/Data/ERData/er_complete_data.csv')

In [16]:
# er_df = ERState_df_with_dup(DAMAN_CSV)
names_df = pd.DataFrame()
names_df['Name'] = er_df['Name']

In [17]:
# dataset = load_dataset('csv', data_files='Models/Data/ERData/er_complete_data.csv', split='train')
dataset = Dataset.from_pandas(names_df)
dataset = dataset.train_test_split(train_size=0.9, seed=42)

In [18]:
dataset['train'].features

{'Name': Value(dtype='string', id=None)}

In [20]:
names_df.shape

(22217062, 1)

MASKED LANGUAGE MODELLING

In [21]:
from transformers import BertConfig, PreTrainedTokenizerFast
from transformers import RobertaConfig
from tokenizers import Tokenizer
from transformers import BertForMaskedLM, RobertaForMaskedLM
import datasets
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
import wandb
import sys
import torch

import pandas as pd

N_EPOCHS = 100
BATCH_SIZE_PER_GPU = 128
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 2e-5

In [22]:
# model_checkpoint = "bert-base-cased"
model_checkpoint = "google/muril-base-cased"
tokenizer_checkpoint = 'Models/ERData/Tokenizer/trained_tokenizer/indiannames-tokenizer/'

In [23]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

tokenizer.pad_token = "[PAD]"
tokenizer.mask_token = "[MASK]"

In [24]:
def tokenize_function(examples):
    return tokenizer(examples["Name"])

In [26]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=2, remove_columns=["Name"])

In [27]:
# set format
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask"])

In [28]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [29]:
from transformers import AutoConfig, AutoModelForMaskedLM

config = AutoConfig.from_pretrained(model_checkpoint)

config.vocab_size=tokenizer.vocab_size
config.max_position_embeddings=100
config.num_attention_heads=12
config.num_hidden_layers=6
config.type_vocab_size=1

print(config)

BertConfig {
  "_name_or_path": "google/muril-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 100,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.13.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 10000
}



In [30]:
model = AutoModelForMaskedLM.from_config(config)

In [31]:
n_train = len(tokenized_datasets["train"])
gpu_count = torch.cuda.device_count()
total_opt_steps = int((n_train * N_EPOCHS) / (BATCH_SIZE_PER_GPU * gpu_count))
WARMUP = int(0.1 * n_train / (BATCH_SIZE_PER_GPU * gpu_count))
SAVE_STEPS = int(total_opt_steps / 20)  # save every 20th of the way
LOGGING_STEPS = int(total_opt_steps / 10)  # log every 10th of the way
EVAL_STEPS = int(total_opt_steps / 10)  # eval every 10th of the way

training_args = TrainingArguments(
    output_dir=  "indiannames-mlm-muril",
    overwrite_output_dir=True,
    num_train_epochs=N_EPOCHS,
    per_gpu_train_batch_size=BATCH_SIZE_PER_GPU,
    per_gpu_eval_batch_size=BATCH_SIZE_PER_GPU,
    warmup_steps=WARMUP,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    save_steps=SAVE_STEPS,
    logging_steps=LOGGING_STEPS,
    eval_steps=EVAL_STEPS,
    evaluation_strategy="steps",
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],  # training dataset
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
)


In [32]:
# wandb logging
wandb.init(project="nameMurilBERT")
run_name = wandb.run.name

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vahini10 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [33]:
# training_args = TrainingArguments(
#     "indiannames-mlm-muril",
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5,
#     weight_decay=0.01,
#         eval_steps=500,
#      num_train_epochs=30,
# #     push_to_hub=False,
# #     push_to_hub_model_id=f"{model_checkpoint}-wikitext2",
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["test"],
#     data_collator=data_collator,
# )

In [34]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 19995355
  Num Epochs = 100
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 7810700
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Using deprecated `--per_gpu_train_batch_size`

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [118]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
***** Running Evaluation *****
  Num examples = 30
  Batch size = 128


Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


Perplexity: 802.78


Save Pretrained Model

In [119]:
trainer.save_model(f"trained_models/{run_name}")

Saving model checkpoint to trained_models/dazzling-lamp-1
Configuration saved in trained_models/dazzling-lamp-1/config.json
Model weights saved in trained_models/dazzling-lamp-1/pytorch_model.bin
